# Topic Modeling  (Section - 1)

### DSCI6003_and_DSCI6004 Final Project

#### Author : Srini Ananthakrishnan
#### Date    : 12/15/2016

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)

### Import GraphLab package

In [2]:
import graphlab as gl
from IPython.display import HTML
from IPython.display import display
from IPython.display import Image
gl.canvas.set_target('ipynb')

### Import pyLDAvis package

In [3]:
import pyLDAvis
import pyLDAvis.graphlab
pyLDAvis.enable_notebook()

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1488392376.log


This non-commercial license of GraphLab Create for academic use is assigned to sanan2@galvanizeu.newhaven.edu and will expire on December 05, 2017.


### Import pre_processing (custom) package

In [4]:
import pre_processing as pp

## Step 1: Data Exploration
**Understand data --> Class balance --> Visualize and Explore**

### Load file and filter reviews

In [5]:
# Reviews file
REVIEWS_FILE = 'music_reviews.json'

In [6]:
# Load jason file to sFrame
sFrame = pp.load_json_file_to_sframe(REVIEWS_FILE)

Finished parsing file /Users/srianant/Programming/ML/GitHub/GalvanizeU/DSCI6004-student/week_8/8_2_final_project/music_reviews.json

Parsing completed. Parsed 100 lines in 1.26969 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 38574 lines. Lines per second: 13103.3

Finished parsing file /Users/srianant/Programming/ML/GitHub/GalvanizeU/DSCI6004-student/week_8/8_2_final_project/music_reviews.json

Parsing completed. Parsed 64706 lines in 3.55595 secs.

In [7]:
# Filter rows with no reviews
reviews = sFrame.filter_by(values = '', column_name = 'reviewText', exclude = True)
# Drop rows where every value is missing
reviews = sFrame.dropna(how='all')

### Peek into reviews data

In [8]:
reviews.show()

### Check for imbalance and balance it

In [9]:
for rating in range(1, 6):
    index = reviews['overall'] == rating
    print (str(rating) + ': Count = '+ str(sum(index)))

1: Count = 2791
2: Count = 3010
3: Count = 6789
4: Count = 16536
5: Count = 35580


In [10]:
reviews.show(view = 'Bar Chart', x = 'overall')

### Let's under_sample and balance the reviews

In [11]:
reviews_balanced = pp.balance_data(reviews,5000)

### Look at balanced reviews

In [12]:
for rating in range(1, 6):
    idx = reviews_balanced['overall'] == rating
    print str(rating) + ': Count = '+ str(sum(idx))

1: Count = 5582
2: Count = 3010
3: Count = 5000
4: Count = 5000
5: Count = 5000


In [13]:
reviews_balanced.show(view = 'Bar Chart', x = 'overall')

## Visualize and explore reviews data

In [14]:
reviews_balanced.show(view = 'Bar Chart', x = 'asin')


## Step 2: Pre-processing
**Tokenize --> Remove stopwords --> Lemmetize**



### Preprocess reviews

In [15]:
reviews_balanced['processedReviews'] = reviews_balanced['reviewText'].apply(pp.preprocess_pipeline)

## Step 3: Build N-Gram models
** Unigram --> Bigram --> Trigram **

### Build Unigram, Bigram and Trigram of reviews

In [16]:
reviews_balanced['Unigram'] =  gl.text_analytics.count_ngrams(reviews_balanced['processedReviews'], n=1)
reviews_balanced['Bigram']  =  gl.text_analytics.count_ngrams(reviews_balanced['processedReviews'], n=2)
reviews_balanced['Trigram'] =  gl.text_analytics.count_ngrams(reviews_balanced['processedReviews'], n=3)

#### Peek into N-Gram models

In [17]:
reviews_balanced

asin,helpful,overall,reviewText,reviewTime,reviewerID
B0021X515S,"[8, 31]",2.0,These guys are talentedbut this is not my type ...,"06 19, 2009",A2R8WPPAH1SEAQ
B00005NKYQ,"[3, 4]",3.0,The concept for this CDis high-concept femin ...,"05 12, 2005",AYOO12C9Y2T95
B00065BYAY,"[4, 7]",4.0,I rushed out to getFantasia's cd in support ...,"01 30, 2005",A35JR4D6FLXYRQ
B000002NG7,"[1, 13]",2.0,When I first heard FaithNo More was putting out ...,"12 30, 2004",A1DIFL0333QPEB
B0000TAZS8,"[4, 6]",2.0,Young Buck flat outsucks. 50 can be good or ...,"11 1, 2005",A39XGSL1OGDVDE
B00004TL26,"[0, 0]",5.0,"OMG, what did i tell yallpeople! Busta wont go ...","01 28, 2003",A3RCTN2UUW6EQ2
B00005NDZC,"[2, 2]",5.0,I bought this album forone song &quot;Hide ...,"02 11, 2003",AGG41GO9FT3FA
B00FAEQ22G,"[0, 0]",5.0,Just a good time tunethat has some catchy ...,"01 21, 2014",A1R4HQIWADCTP0
B000BY8278,"[3, 4]",4.0,"As always, The IsleyBros. return as ...","05 12, 2006",A1QEWOSV05RYEO
B00005TPKC,"[1, 2]",4.0,"I own all of Alanis'sstudio albums, and ...","07 12, 2004",A20OAPE0RCEJ9P


In [18]:
# Drop rows where every value is missing on balanced data
reviews_balanced = reviews_balanced.dropna()

## Step 4: Apply LDA (Latent Dirchilet Allocation)
** Create models using GraphLab LDA to identify topics**
## Step 5: Visualize Topics
**Visualize Top 5 most relevant words for each topic**

### Create Unigram Topic Model

In [19]:
unigram_topic_model = gl.topic_model.create(reviews_balanced['Unigram'], num_topics=10, num_iterations=200)

Learning a topic model

Number of documents     23440

Vocabulary size     22076

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 14.48s        | 3.39842e+06    | 0               |

| 180       | 15.29s        | 4.23335e+06    | 0               |

| 190       | 16.03s        | 4.82867e+06    | 0               |

| 200       | 16.81s        | 4.41613e+06    | 0               |

+-----------+---------------+----------------+-----------------+

### Unigram: Top 5 most relevant words for each topic

In [20]:
unigram_topic_model.get_topics().print_rows(100)

+-------+---------+------------------+
| topic |   word  |      score       |
+-------+---------+------------------+
|   0   |    cd   | 0.0578316217822  |
|   0   |   fan   | 0.0334037560094  |
|   0   |  first  | 0.0307929153187  |
|   0   |  album  | 0.0271317364191  |
|   0   |  heard  |  0.027101726756  |
|   1   |  album  | 0.0738048342734  |
|   1   |   song  | 0.0664042468882  |
|   1   |  great  | 0.0328976504592  |
|   1   |   time  | 0.0195532842292  |
|   1   |   good  | 0.0189414246422  |
|   2   |   rap   | 0.0225101713448  |
|   2   |  album  | 0.0181807585353  |
|   2   |   hop   | 0.0147116777585  |
|   2   |  rapper | 0.0147116777585  |
|   2   |   hip   |  0.014656172466  |
|   3   |  music  | 0.0435222064928  |
|   3   |   one   | 0.0233672049332  |
|   3   |  artist | 0.0211477553567  |
|   3   |   like  | 0.0140695107613  |
|   3   |   new   | 0.0103804256544  |
|   4   |   dont  | 0.0310474881308  |
|   4   |   like  | 0.0302677632871  |
|   4   |   get   | 0.025

### Visualize Unigram Model

In [21]:
pyLDAvis.graphlab.prepare(unigram_topic_model, reviews_balanced['Unigram'])

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      10.392904        1       1 -0.141160  0.010890
2      10.338349        1       2 -0.174815 -0.140193
5      10.209788        1       3  0.088851 -0.014972
9      10.120500        1       4  0.250785 -0.243811
4       9.984727        1       5 -0.219163 -0.118234
1       9.972126        1       6 -0.035023  0.101463
8       9.949457        1       7  0.009697  0.102940
3       9.755527        1       8  0.117453 -0.032706
7       9.743859        1       9  0.153875  0.180485
0       9.532763        1      10 -0.050500  0.154137, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
19255  Default  3629.000000          quot  3629.000000  30.0000  30.0000
12803  Default  9421.000000         album  9421.000000  29.0000  29.0000
21385  Default  3618.000000          song  3618.000000  28.0000  28.0000
6714   Default  2905.000000            cd  2905.000000  27.0000  27.0000
3128   Default  1907.000000         first  1907.000000  26.0000  26.0000
453    Default  3259.000000           one  3259.000000  25.0000  25.0000
46     Default  2162.000000         music  2162.000000  24.0000  24.0000
1816   Default  1279.000000          year  1279.000000  23.0000  23.0000
12456  Default  1376.000000          band  1376.000000  22.0000  22.0000
17551  Default  1371.000000         great  1371.000000  21.0000  21.0000
19999  Default  2195.000000          good  2195.000000  20.0000  20.0000
14174  Default   977.000000          rock   977.000000  19.0000  19.0000
21389  Default  1251.000000           fan  1251.000000  18.0000  18.0000
14547  Default  1171.000000          dont  1171.000000  17.0000  17.0000
12763  Default   870.000000         heard   870.000000  16.0000  16.0000
16811  Default  1600.000000          time  1600.000000  15.0000  15.0000
3197   Default  1273.000000           get  1273.000000  14.0000  14.0000
21577  Default   916.000000           hit   916.000000  13.0000  13.0000
20790  Default   752.000000           rap   752.000000  12.0000  12.0000
5915   Default   741.000000             2   741.000000  11.0000  11.0000
19709  Default  1089.000000         track  1089.000000  10.0000  10.0000
17273  Default  1345.000000        really  1345.000000   9.0000   9.0000
21197  Default   693.000000        review   693.000000   8.0000   8.0000
16576  Default   710.000000         debut   710.000000   7.0000   7.0000
13294  Default  1177.000000          love  1177.000000   6.0000   6.0000
17000  Default   669.000000        artist   669.000000   5.0000   5.0000
16659  Default   831.000000       release   831.000000   4.0000   4.0000
2650   Default   672.000000          star   672.000000   3.0000   3.0000
21995  Default   853.000000          know   853.000000   2.0000   2.0000
8482   Default   997.000000            im   997.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
21389  Topic10  1028.497328           fan  1251.665382   2.1541  -3.3991
6075   Topic10   131.299497          went   150.257719   2.2156  -5.4575
17430  Topic10    74.935885     surprised    83.389243   2.2435  -6.0183
152    Topic10   139.615440        saying   161.222461   2.2065  -5.3960
945    Topic10   149.779370        couple   176.280713   2.1875  -5.3258
2287   Topic10   139.615440      reviewer   163.808379   2.1906  -5.3960
2919   Topic10   130.375504           yes   152.570171   2.1932  -5.4645
10575  Topic10    87.871796      recently    99.904269   2.2221  -5.8591
6714   Topic10  1780.628156            cd  2905.355376   1.8608  -2.8502
7475   Topic10   103.579688          read   120.291661   2.2009  -5.6946
3128   Topic10   948.109881         first  1907.740400   1.6512  -3.4805
19851  Topic10   116.515599  disappointed   143.754648   2.1404  -5.5769
20177  Topic10   332.730

### Create Bigram Topic Model

In [22]:
bigram_topic_model = gl.topic_model.create(reviews_balanced['Bigram'], num_topics=10, num_iterations=200)

Learning a topic model

Number of documents     23440

Vocabulary size    179001

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 14.09s        | 4.81316e+06    | 0               |

| 180       | 14.97s        | 3.7262e+06     | 0               |

| 190       | 15.77s        | 3.9885e+06     | 0               |

| 200       | 16.66s        | 4.01287e+06    | 0               |

+-----------+---------------+----------------+-----------------+

### Bigram: Top 5 most relevant words for each topic

In [23]:
bigram_topic_model.get_topics().print_rows(100)

+-------+-----------------+------------------+
| topic |       word      |      score       |
+-------+-----------------+------------------+
|   0   |      jay z      | 0.00497376187878 |
|   0   |     year ago    | 0.00217193921941 |
|   0   |    new album    | 0.00208767387627 |
|   0   |    album good   | 0.00198234219734 |
|   0   |   studio album  |  0.001624214489  |
|   1   |    good song    | 0.00279005808603 |
|   1   |    long time    | 0.00272717172797 |
|   1   |   pretty good   | 0.00226600510218 |
|   1   |      wan na     | 0.00178387635703 |
|   1   |    sound like   | 0.00151136880543 |
|   2   |       r b       | 0.00318759339905 |
|   2   |    sound like   |  0.001990375221  |
|   2   |   second album  | 0.0015200395082  |
|   2   |   debut album   | 0.00147728171613 |
|   2   |    album came   | 0.00130625054783 |
|   3   |    quot quot    | 0.00642113687658 |
|   3   |    elton john   | 0.00265609987429 |
|   3   |   one favorite  | 0.00232691631125 |
|   3   |    

### Visualize Bigram Model

In [24]:
pyLDAvis.graphlab.prepare(bigram_topic_model, reviews_balanced['Bigram'])

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      10.177108        1       1  0.172248  0.028030
1      10.102586        1       2  0.067554 -0.098087
7      10.073647        1       3 -0.066096 -0.111607
8      10.051325        1       4 -0.057678 -0.063809
9      10.017863        1       5 -0.052348  0.141311
5       9.994957        1       6  0.062082  0.007839
0       9.951063        1       7 -0.028154  0.044122
4       9.908580        1       8 -0.049651 -0.020722
6       9.899436        1       9 -0.037218  0.051051
2       9.823435        1      10 -0.010738  0.021872, topic_info=       Category        Freq               Term       Total  loglift  logprob
term                                                                        
130993  Default  318.000000            hip hop  318.000000  30.0000  30.0000
89530   Default  196.000000          quot quot  196.000000  29.0000  29.0000
24629   Default  161.000000              jay z  161.000000  28.0000  28.0000
168587  Default  138.000000         best album  138.000000  27.0000  27.0000
64374   Default  162.000000          good song  162.000000  26.0000  26.0000
24493   Default  178.000000        debut album  178.000000  25.0000  25.0000
92202   Default  104.000000            ja rule  104.000000  24.0000  24.0000
62655   Default   99.000000             g unit   99.000000  23.0000  23.0000
60932   Default  145.000000           one best  145.000000  22.0000  22.0000
70731   Default   98.000000                r b   98.000000  21.0000  21.0000
98278   Default   86.000000          long time   86.000000  20.0000  20.0000
49675   Default   91.000000         elton john   91.000000  19.0000  19.0000
62920   Default  127.000000       greatest hit  127.000000  18.0000  18.0000
84199   Default  152.000000         sound like  152.000000  17.0000  17.0000
89980   Default   71.000000         first time   71.000000  16.0000  16.0000
107137  Default   74.000000       one favorite   74.000000  15.0000  15.0000
158532  Default   89.000000        pretty good   89.000000  14.0000  14.0000
3625    Default   98.000000             5 star   98.000000  13.0000  13.0000
26678   Default   73.000000        worst album   73.000000  12.0000  12.0000
50592   Default  118.000000          dont know  118.000000  11.0000  11.0000
64985   Default  112.000000         good album  112.000000  10.0000  10.0000
68949   Default  135.000000            50 cent  135.000000   9.0000   9.0000
19236   Default   90.000000         album ever   90.000000   8.0000   8.0000
157867  Default   84.000000           year ago   84.000000   7.0000   7.0000
25820   Default   68.000000         album quot   68.000000   6.0000   6.0000
72601   Default   82.000000        great album   82.000000   5.0000   5.0000
146783  Default  166.000000        first album  166.000000   4.0000   4.0000
150618  Default   80.000000          new album   80.000000   3.0000   3.0000
138664  Default   69.000000         ever heard   69.000000   2.0000   2.0000
6551    Default   75.000000         album good   75.000000   1.0000   1.0000
...         ...         ...                ...         ...      ...      ...
60280   Topic10   10.128470            album 1   11.315337   2.2096  -7.9743
126347  Topic10   38.437857         album came   45.863937   2.1438  -6.6406
107071  Topic10    8.870275         aimee mann   10.058355   2.1947  -8.1069
150648  Topic10   12.015762        eminem show   13.826231   2.1801  -7.8034
86085   Topic10   36.550564         ever since   44.606151   2.1212  -6.6909
80034   Topic10   21.452225        lead singer   25.821451   2.1350  -7.2238
3885    Topic10   18.935835          ive never   22.627929   2.1423  -7.3486
78770   Topic10    8.241177           c murder    9.428044   2.1859  -8.1805
68557   Topic10   17.048542           great cd   21.354466   2.0952  -7.4536
154156  Topic10   20.823127            love cd   27.006076   2.0604  -7.253

### Create Trigram Topic Model

In [25]:
trigram_topic_model = gl.topic_model.create(reviews_balanced['Trigram'], num_topics=10, num_iterations=200)

Learning a topic model

Number of documents     23440

Vocabulary size    231831

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 11.13s        | 4.61727e+06    | 0               |

| 180       | 11.84s        | 4.14311e+06    | 0               |

| 190       | 12.47s        | 4.12204e+06    | 0               |

| 200       | 13.12s        | 4.92768e+06    | 0               |

+-----------+---------------+----------------+-----------------+

### Trigram: Top 5 most relevant words for each topic

In [26]:
trigram_topic_model.get_topics().print_rows(100)

+-------+---------------------+-------------------+
| topic |         word        |       score       |
+-------+---------------------+-------------------+
|   0   |   let start saying  | 0.000611192887365 |
|   0   |    notorious b g    | 0.000414667843196 |
|   0   |       r amp b       | 0.000336057825529 |
|   0   |   breath fresh air  | 0.000336057825529 |
|   0   |    hip hop album    | 0.000296752816695 |
|   1   |    red hot chili    | 0.000589651517911 |
|   1   |      im gon na      | 0.000354574500804 |
|   1   |    one hit wonder   | 0.000334984749378 |
|   1   |  one favorite album | 0.000334984749378 |
|   1   |   hard knock life   | 0.000276215495101 |
|   2   |    hip hop album    | 0.000356973104739 |
|   2   |  greatest hit album | 0.000337250833759 |
|   2   |    one best album   | 0.000258361749838 |
|   2   |      n da hood      | 0.000258361749838 |
|   2   |     album jay z     | 0.000238639478858 |
|   3   |  self titled debut  | 0.000590414879239 |
|   3   |   

### Visualize Trigram Model

In [27]:
pyLDAvis.graphlab.prepare(trigram_topic_model, reviews_balanced['Trigram'])

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      10.058766        1       1  0.128074 -0.050749
0      10.038997        1       2  0.057041  0.009995
1      10.028186        1       3 -0.030348  0.153028
7      10.018308        1       4  0.038732  0.005237
3      10.017270        1       5 -0.043447 -0.014878
4       9.997678        1       6 -0.018758  0.020473
9       9.976817        1       7 -0.109557 -0.091175
6       9.965479        1       8  0.002163  0.009143
2       9.954238        1       9 -0.001121 -0.038590
5       9.944260        1      10 -0.022778 -0.002484, topic_info=       Category       Freq                 Term      Total  loglift  logprob
term                                                                        
191248  Default  24.000000         get rich die  24.000000  30.0000  30.0000
108272  Default  20.000000             1 2 star  20.000000  29.0000  29.0000
33068   Default  19.000000   jagged little pill  19.000000  28.0000  28.0000
187761  Default  33.000000              r amp b  33.000000  27.0000  27.0000
104248  Default  31.000000       dont get wrong  31.000000  26.0000  26.0000
109300  Default  17.000000     let start saying  17.000000  25.0000  25.0000
114666  Default  23.000000         mary j blige  23.000000  24.0000  24.0000
73223   Default  18.000000        red hot chili  18.000000  23.0000  23.0000
125266  Default  17.000000       rich die tryin  17.000000  22.0000  22.0000
97024   Default  18.000000    self titled debut  18.000000  21.0000  21.0000
120689  Default  16.000000       ive ever heard  16.000000  20.0000  20.0000
110269  Default  24.000000       one best album  24.000000  19.0000  19.0000
148817  Default  19.000000        hip hop album  19.000000  18.0000  18.0000
101819  Default  13.000000        notorious b g  13.000000  17.0000  17.0000
53321   Default  15.000000          rock n roll  15.000000  16.0000  16.0000
192564  Default  14.000000   greatest hit album  14.000000  15.0000  15.0000
145893  Default  10.000000           got ta say  10.000000  14.0000  14.0000
157064  Default  17.000000            im gon na  17.000000  13.0000  13.0000
15210   Default  17.000000     hot chili pepper  17.000000  12.0000  12.0000
162886  Default   9.000000     breath fresh air   9.000000  11.0000  11.0000
72310   Default   9.000000   one favorite album   9.000000  10.0000  10.0000
187022  Default   9.000000    every single song   9.000000   9.0000   9.0000
182474  Default   9.000000     dont waste money   9.000000   8.0000   8.0000
97533   Default  11.000000       one hit wonder  11.000000   7.0000   7.0000
227494  Default  12.000000      one worst album  12.000000   6.0000   6.0000
189943  Default   9.000000             4 5 star   9.000000   5.0000   5.0000
14228   Default   8.000000          10 x factor   8.000000   4.0000   4.0000
200899  Default   9.000000       song like quot   9.000000   3.0000   3.0000
3225    Default   9.000000          hip hop fan   9.000000   2.0000   2.0000
31016   Default  10.000000      hard knock life  10.000000   1.0000   1.0000
...         ...        ...                  ...        ...      ...      ...
43833   Topic10   2.228391         classic 1 10   2.717632   2.1097  -9.4191
200113  Topic10   2.228391            one top 5   2.717632   2.1097  -9.4191
63902   Topic10   2.228391      two bonus track   2.717632   2.1097  -9.4191
30160   Topic10   2.228391        cant rap cant   2.717632   2.1097  -9.4191
26477   Topic10   2.228391          best cd ive   2.717632   2.1097  -9.4191
152901  Topic10   2.228391      quot angel quot   2.717632   2.1097  -9.4191
88615   Topic10   2.228391      7 10 production   2.717632   2.1097  -9.4191
139548  Topic10   2.228391     give lousy album   2.717632   2.1097  -9.4191
223752  Topic10   4.402431        member g unit   5.436414   2.0972  -8.7382
114666  Topic10  17.990183         mary j blige  23.350716   2.0474  -7.330

## Challenges:
** Handling large datasets (> 1-million reviews) : **  
**- Visualization with large dataset using pyLDAvis was not possible to render in iPython notebook **  
**- Could not build models due to limited processing power of my computer **  

## Conclusion:
** - Imbalanced review data caused topic modelling find topics more biased towards positive reviews. **  
** - Bigram/Trigram model even though had large feature space was able to find topics both positive and negative reviews more clearly. **